# CO₂-Based Closed-Loop Geothermal Well-Field Optimization

This notebook demonstrates well-field optimization for a CO₂-based closed-loop geothermal pattern:
- 1 center producer (P0) at (0, 0)
- 3 injectors (I1-I3) on an inner ring
- 4 producers (P1-P4) on an outer ring

**Goal**: Minimize pressure-drop non-uniformity (utilization factor) under fixed flow allocation,
while also estimating thermal lifetime using a conical frustum reservoir volume model.

## Optimization Variables
- `R_in`: Inner ring radius for injectors [m]
- `R_out`: Outer ring radius for producers [m]
- `θ0`: Global rotation angle for outer ring [rad]
- `ε1, ε2, ε3`: Deviations from 90° increments for outer producers [rad]

## Objective Function
```
J = w1·CV_inj + w2·CV_prod + w4·CV_tof - w3·(τ/τ_ref) + Penalty
```

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Check for CoolProp
try:
    import CoolProp
    print(f"CoolProp version: {CoolProp.__version__}")
except ImportError:
    raise ImportError(
        "CoolProp is required for this notebook.\n"
        "Install with: pip install CoolProp\n"
        "Or: conda install -c conda-forge coolprop"
    )

# Import wellfield package
import sys
sys.path.insert(0, '.')

from wellfield import (
    Config,
    compute_well_coordinates,
    compute_all_well_positions,
    compute_objective,
    run_optimization,
    plots,
)
from wellfield.objective import evaluate_solution, print_solution_summary
from wellfield.geometry import get_default_initial_guess, x_to_params
from wellfield.hydraulics import get_mean_fluid_properties
from wellfield.thermal import get_thermal_metrics

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150

## 1. Configuration

All parameters are centralized in the `Config` class with exact defaults from the specification.

In [ ]:
# Create configuration with default values
config = Config()

print("=" * 60)
print("CONFIGURATION SUMMARY")
print("=" * 60)

print("\n--- Hydraulics ---")
print(f"  Total mass flow rate: {config.M_DOT_TOTAL} kg/s")
print(f"  Permeability:         {config.K_PERM:.2e} m² ({config.K_PERM/1e-15:.0f} mD)")
print(f"  Reservoir thickness:  {config.H_THICK} m")
print(f"  Well diameter:        {config.D_WELL} m")
print(f"  Injection pressure:   {config.P_INJ/1e6:.1f} MPa")
print(f"  Production pressure:  {config.P_PROD/1e6:.1f} MPa")
print(f"  Injection temp:       {config.T_INJ_C}°C")
print(f"  Production temp:      {config.T_PROD_C}°C")

print("\n--- Thermal ---")
print(f"  Porosity:             {config.POROSITY}")
print(f"  Rock density:         {config.RHO_ROCK} kg/m³")
print(f"  Rock heat capacity:   {config.CP_ROCK} J/(kg·K)")
print(f"  Reservoir temp:       {config.T_RES_C}°C")
print(f"  Working temp:         {config.T_WORK_C}°C")

print("\n--- Geometry Constraints ---")
print(f"  Min well spacing:     {config.S_MIN} m")
print(f"  Min radial gap:       {config.DELTA_R_MIN} m")
print(f"  Min angle increment:  {config.DELTA_THETA_MIN_DEG}°")
print(f"  Max epsilon:          {config.EPS_MAX_DEG}°")

print("\n--- Objective Weights ---")
print(f"  w1 (CV_inj):          {config.W1}")
print(f"  w2 (CV_prod):         {config.W2}")
print(f"  w3 (lifetime):        {config.W3}")
print(f"  w4 (CV_tof):          {config.W4}")
print(f"  τ_ref:                {config.TAU_REF} years")

print("\n--- Optimization ---")
print(f"  DE population size:   {config.DE_POPSIZE}")
print(f"  DE max iterations:    {config.DE_MAXITER}")
print(f"  Random seed:          {config.DE_SEED}")

## 2. Fluid Properties from CoolProp

In [ ]:
# Get CO2 properties at mean operating conditions
props = get_mean_fluid_properties(config)

print("CO₂ Properties at Mean Conditions")
print(f"  T_mean = {config.T_MEAN_K:.1f} K ({config.T_MEAN_K - 273.15:.1f}°C)")
print(f"  P_mean = {config.P_MEAN/1e6:.2f} MPa")
print(f"  \n  Viscosity (μ):     {props['mu']:.2e} Pa·s")
print(f"  Density (ρ):       {props['rho']:.1f} kg/m³")
print(f"  Heat capacity (cp): {props['cp']:.1f} J/(kg·K)")

## 3. Initial Layout Visualization

In [ ]:
# Get initial guess
x0 = get_default_initial_guess(config)
R_in, R_out, theta0, eps1, eps2, eps3 = x_to_params(x0)

print("Initial Guess:")
print(f"  R_in  = {R_in:.1f} m")
print(f"  R_out = {R_out:.1f} m")
print(f"  θ0    = {np.rad2deg(theta0):.1f}°")
print(f"  ε1, ε2, ε3 = {np.rad2deg(eps1):.1f}°, {np.rad2deg(eps2):.1f}°, {np.rad2deg(eps3):.1f}°")

# Compute coordinates
coords_initial = compute_well_coordinates(R_in, R_out, theta0, eps1, eps2, eps3)

# Plot initial layout
fig = plots.plot_well_layout(coords_initial, config)
plt.title('Initial Well Layout', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Evaluate initial solution
print("\nInitial Solution Metrics:")
metrics_initial = evaluate_solution(x0, config)
print(f"  J        = {metrics_initial['J']:.4f}")
print(f"  CV_inj   = {metrics_initial['CV_inj']:.4f}")
print(f"  CV_prod  = {metrics_initial['CV_prod']:.4f}")
print(f"  CV_tof   = {metrics_initial['CV_tof']:.4f}")
print(f"  τ        = {metrics_initial['tau_years']:.1f} years")
print(f"  Penalty  = {metrics_initial['penalty']:.2e}")

## 4. Run Optimization

Using scipy's `differential_evolution` with fixed random seed for reproducibility.

In [ ]:
# Run the optimizer
x_best, opt_info = run_optimization(config, verbose=True)

## 5. Results Summary

In [ ]:
# Print detailed summary
print_solution_summary(x_best, config)

In [ ]:
# Get optimized coordinates
R_in_opt, R_out_opt, theta0_opt, eps1_opt, eps2_opt, eps3_opt = x_to_params(x_best)
coords_opt = compute_well_coordinates(R_in_opt, R_out_opt, theta0_opt, eps1_opt, eps2_opt, eps3_opt)

print("\nWell Coordinates (Optimized):")
print("-" * 40)
for name, (x, y) in coords_opt.items():
    print(f"  {name}: ({x:8.1f}, {y:8.1f}) m")

## 6. Required Plots

### 6.1 Well Layout Plot

In [ ]:
# Well layout with labels
fig = plots.plot_well_layout(coords_opt, config)
plt.title('Optimized Well Layout', fontsize=14, fontweight='bold')
plt.savefig('well_layout.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.2 Pressure Map

In [ ]:
# Pressure contour map
fig = plots.plot_pressure_map(coords_opt, config)
plt.savefig('pressure_map.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.3 Streamlines

In [ ]:
# Streamlines with velocity field
fig = plots.plot_streamlines(coords_opt, config)
plt.savefig('streamlines.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.4 Metrics Bar Plots

In [ ]:
# Evaluate optimized solution for metrics
metrics_opt = evaluate_solution(x_best, config)

# Pressure drops and breakthrough times
fig = plots.plot_metrics_bars(metrics_opt)
plt.savefig('metrics_bars.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.5 Convergence Plot

In [ ]:
# Optimization convergence
fig = plots.plot_convergence(opt_info['history'])
plt.savefig('convergence.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Comparison: Initial vs Optimized

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Initial layout
ax = axes[0]
for name in ['I1', 'I2', 'I3']:
    x, y = coords_initial[name]
    ax.scatter(x, y, c='blue', s=150, marker='^', edgecolors='black', linewidths=1.5, zorder=3)
    ax.annotate(name, (x, y), xytext=(8, 8), textcoords='offset points', fontsize=10, color='blue')

for name in ['P0', 'P1', 'P2', 'P3', 'P4']:
    x, y = coords_initial[name]
    marker = 'o' if name == 'P0' else 's'
    ax.scatter(x, y, c='red', s=150, marker=marker, edgecolors='black', linewidths=1.5, zorder=3)
    ax.annotate(name, (x, y), xytext=(8, -12), textcoords='offset points', fontsize=10, color='red')

ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title(f'Initial Layout\nJ = {metrics_initial["J"]:.4f}', fontsize=12, fontweight='bold')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlim(-3000, 3000)
ax.set_ylim(-3000, 3000)

# Optimized layout
ax = axes[1]
for name in ['I1', 'I2', 'I3']:
    x, y = coords_opt[name]
    ax.scatter(x, y, c='blue', s=150, marker='^', edgecolors='black', linewidths=1.5, zorder=3)
    ax.annotate(name, (x, y), xytext=(8, 8), textcoords='offset points', fontsize=10, color='blue')

for name in ['P0', 'P1', 'P2', 'P3', 'P4']:
    x, y = coords_opt[name]
    marker = 'o' if name == 'P0' else 's'
    ax.scatter(x, y, c='red', s=150, marker=marker, edgecolors='black', linewidths=1.5, zorder=3)
    ax.annotate(name, (x, y), xytext=(8, -12), textcoords='offset points', fontsize=10, color='red')

ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title(f'Optimized Layout\nJ = {metrics_opt["J"]:.4f}', fontsize=12, fontweight='bold')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlim(-3000, 3000)
ax.set_ylim(-3000, 3000)

plt.tight_layout()
plt.savefig('comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Thermal Analysis

In [ ]:
# Get detailed thermal metrics
thermal = get_thermal_metrics(R_in_opt, R_out_opt, config)

print("Thermal Analysis (Conical Frustum Model)")
print("=" * 50)
print(f"\n  Reservoir volume:    {thermal['V_res']/1e9:.3f} × 10⁹ m³")
print(f"  Rock mass:           {thermal['m_rock']/1e12:.3f} × 10¹² kg")
print(f"  Water mass:          {thermal['m_water']/1e9:.3f} × 10⁹ kg")
print(f"\n  Total heat content:  {thermal['Q_res_GJ']:.1f} GJ")
print(f"                       {thermal['Q_res_GJ']/1e6:.3f} × 10⁶ GJ")
print(f"\n  Heat extraction rate: {thermal['Q_dot_MW']:.2f} MW")
print(f"  CO₂ heat capacity:    {thermal['cp_co2']:.1f} J/(kg·K)")
print(f"\n  Thermal lifetime:    {thermal['tau_years']:.1f} years")

## 9. Summary Table

In [ ]:
# Create summary comparison table
print("\n" + "=" * 70)
print("OPTIMIZATION RESULTS SUMMARY")
print("=" * 70)

print("\n{:<25} {:>15} {:>15} {:>12}".format("Parameter", "Initial", "Optimized", "Change"))
print("-" * 70)

# Geometry
R_in_init, R_out_init, _, _, _, _ = x_to_params(x0)
print(f"{'R_in [m]':<25} {R_in_init:>15.1f} {R_in_opt:>15.1f} {R_in_opt - R_in_init:>+12.1f}")
print(f"{'R_out [m]':<25} {R_out_init:>15.1f} {R_out_opt:>15.1f} {R_out_opt - R_out_init:>+12.1f}")

# Objective
print(f"{'J (objective)':<25} {metrics_initial['J']:>15.4f} {metrics_opt['J']:>15.4f} {metrics_opt['J'] - metrics_initial['J']:>+12.4f}")
print(f"{'CV_inj':<25} {metrics_initial['CV_inj']:>15.4f} {metrics_opt['CV_inj']:>15.4f} {metrics_opt['CV_inj'] - metrics_initial['CV_inj']:>+12.4f}")
print(f"{'CV_prod':<25} {metrics_initial['CV_prod']:>15.4f} {metrics_opt['CV_prod']:>15.4f} {metrics_opt['CV_prod'] - metrics_initial['CV_prod']:>+12.4f}")
print(f"{'CV_tof':<25} {metrics_initial['CV_tof']:>15.4f} {metrics_opt['CV_tof']:>15.4f} {metrics_opt['CV_tof'] - metrics_initial['CV_tof']:>+12.4f}")
print(f"{'τ [years]':<25} {metrics_initial['tau_years']:>15.1f} {metrics_opt['tau_years']:>15.1f} {metrics_opt['tau_years'] - metrics_initial['tau_years']:>+12.1f}")

print("-" * 70)
improvement = (metrics_initial['J'] - metrics_opt['J']) / abs(metrics_initial['J']) * 100
print(f"\nOverall improvement: {improvement:.1f}%")

## 10. Summary Figure

In [ ]:
# Create comprehensive summary figure
fig = plots.create_summary_figure(x_best, metrics_opt, config)
plt.savefig('summary_figure.png', dpi=150, bbox_inches='tight')
plt.show()

## Conclusion

The optimization successfully found a well layout that:
1. **Minimizes pressure-drop non-uniformity** among injectors and producers (lower CV values)
2. **Balances breakthrough times** across all producers (lower CV_tof)
3. **Maximizes thermal lifetime** through optimal radii selection
4. **Satisfies all geometric constraints** (minimum spacing, radial gap, angle increments)

The results demonstrate the physics-based forward model approach, where every iteration evaluates the actual pressure field and breakthrough characteristics rather than using geometric shortcuts.